In [1]:
import mxnet as mx
# import apollocaffe
# out = mx.symbol.load("googlenet-symbol.json")

In [2]:
pretrained_cnn = mx.nd.load("googlenet-0001.params")


In [3]:
out = mx.symbol.load("googlenet-symbol.json")
arg_shapes, out_shapes, _ = out.infer_shape(data=(1,3,480,640),lstm_hidden_seed=(300,250),lstm_mem_seed=(300,250))
print out_shapes

[(300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 2L), (300L, 2L), (300L, 2L), (300L, 2L), (300L, 2L), (1L, 1024L, 15L, 20L)]


In [4]:
arg_names = out.list_arguments()
# for name in arg_names:
#     if name == "data":
#         continue
#     key = "arg:" + name
#     if key in pretrained:
#         pretrained[key].copyto(arg_dict[name])
#     else:
#         print ("SKIP arguments %s" % name)
# print arg_names
# arg_names = arg_names + ["rois"]
# arg_shapes = arg_shapes + [(1,5)]

In [5]:
arg_dict = dict(zip(arg_names, [mx.nd.zeros(shape,ctx=mx.gpu(0)) for shape in arg_shapes]))



In [6]:
import pandas as pd
lstm_content = pd.read_hdf("lstm.h5","lstm")
param_name = list(lstm_content["param_name"])
param_ndarray = list(lstm_content["param_ndarray"])

param_dict = dict(zip(param_name,param_ndarray))
for k,v in param_dict.items():
    print k,v.shape

ip_conf3_bias (2,)
ip_bbox_unscaled4_weight (4, 250)
ip_bbox_unscaled3_weight (4, 250)
output_gate_weight (250, 1274)
ip_bbox_unscaled2_bias (4,)
ip_conf4_weight (2, 250)
ip_conf3_weight (2, 250)
forget_gate_weight (250, 1274)
ip_bbox_unscaled4_bias (4,)
input_value_weight (250, 1274)
ip_conf0_weight (2, 250)
ip_conf2_weight (2, 250)
ip_conf1_weight (2, 250)
ip_conf4_bias (2,)
ip_conf0_bias (2,)
input_gate_weight (250, 1274)
ip_bbox_unscaled0_weight (4, 250)
ip_bbox_unscaled1_bias (4,)
ip_conf2_bias (2,)
ip_conf1_bias (2,)
ip_bbox_unscaled0_bias (4,)
ip_bbox_unscaled3_bias (4,)
ip_bbox_unscaled2_weight (4, 250)
ip_bbox_unscaled1_weight (4, 250)


In [7]:
for name in arg_names:
    if name in ["data","lstm_mem_seed","lstm_hidden_seed"]:
        continue
    key = "arg:" + name
    if key in pretrained_cnn:
        pretrained_cnn[key].copyto(arg_dict[name])
    elif "input_value" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["input_value_weight"]).copyto(arg_dict[name])
    elif "output_gate" in name:
#         print arg_dict[name].shape
#         print param_dict["output_gate_weight"].shape
        mx.nd.array(param_dict["output_gate_weight"]).copyto(arg_dict[name])
    elif "input_gate" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["input_gate_weight"]).copyto(arg_dict[name])
    elif "forget_value" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["forget_gate_weight"]).copyto(arg_dict[name])
    elif name in param_name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict[name]).copyto(arg_dict[name])
    else:
        print ("SKIP arguments %s" % name)

In [8]:
# model = mx.model.FeedForward(out,num_epoch=100,learning_rate=0.01)
mx.model.save_checkpoint("listen_mxnet_first",epoch=0,symbol=out,arg_params=arg_dict,aux_params={})


In [9]:
from utils import (annotation_jitter, image_to_h5_mx,
                   annotation_to_h5, load_data_mean_mx)
from utils.annolist import AnnotationLib as al
import time
import os
from scipy.misc import imread,imsave
import random
from utils.annolist import AnnotationLib as al
# from train import load_idl, forward
from utils import (annotation_jitter,
                   annotation_to_h5, Rect, stitch_rects)



def load_idl(idlfile, data_mean, jitter=True):
    """Take the idlfile, data mean and net configuration and create a generator
    that outputs a jittered version of a random image from the annolist
    that is mean corrected."""

    annolist = al.parse(idlfile)
    annos = [x for x in annolist]
    for anno in annos:
        anno.imageName = os.path.join(
            os.path.dirname(os.path.realpath(idlfile)), anno.imageName)
    while True:
        random.shuffle(annos)
        for anno in annos:
#             if jitter:
#                 jit_image, jit_anno = annotation_jitter(
#                     anno, target_width=640,
#                     target_height=480)
#             else:
            jit_image = imread(anno.imageName)
#                 jit_anno = anno
#             image = image_to_h5_mx(jit_image, data_mean, image_scaling=1.0)
#             boxes, box_flags = annotation_to_h5(
#                 jit_anno,640, 480,
#                 64, 5)
#             yield { "image": jit_image,'anno': jit_anno}
            yield jit_image
        break

image_mean = load_data_mean_mx("./data/brainwash_mean.npy", 640,480, image_scaling=1.0)

In [10]:
# input_gen = load_idl("./data/brainwash/brainwash_train.idl",image_mean, jitter = False)

In [11]:
from collections import namedtuple
import cv2
max_len = 5

executor = out.bind(ctx=mx.gpu(), args=arg_dict)
e2eExcutor = namedtuple("e2eExcutor",['executor','data','lstm_mem_seed','lstm_hidden_seed','bbox', 'conf',"fm",'arg_dict'])

timeExcutor = e2eExcutor(executor = executor, 
                         data = arg_dict["data"], 
                         lstm_mem_seed = arg_dict["lstm_mem_seed"], 
                         lstm_hidden_seed = arg_dict["lstm_hidden_seed"],
                         bbox = executor.outputs[:max_len],
                         conf = executor.outputs[max_len:],
                         fm = executor.outputs[2*max_len],
                         arg_dict=arg_dict)


In [12]:
# def overlap_union(x1,y1,x2,y2,x3,y3,x4,y4):
#     SI = max(0, min(x2,x4)-max(x1,x3)) * max(0, min(y2,y4)-max(y1,y3))
#     SU = (x2-x1)*(y2-y1) + (x4-x3)*(y4-y3) - SI + 0.0
#     return SI/SU

def generate_rois(batch_idx, rect, proposals):
#     proposals =  []
#     for rect in acc_rects:
    left_topx = rect.cx - int(rect.width)
    left_topy = rect.cy - int(rect.height/2)
    proposals.append([batch_idx,left_topx,left_topy,left_topx+rect.width*2,left_topy+rect.height*5])
#     return proposals


In [13]:
rois = mx.symbol.Variable(name = "rois")
fm = mx.symbol.Variable(name =  "fm")
roi_pool = mx.symbol.ROIPooling(data = fm, rois = rois, pooled_size = (7,7), spatial_scale = 0.03125)
pool5 = mx.symbol.Pooling(data = roi_pool, pool_type="max", kernel=(7,7), stride=(1,1), name= "pool5")
print pool5.list_arguments()

rois_dict = dict(zip(["rois","fm"], [mx.nd.zeros((1,5),ctx=mx.gpu(0)),mx.nd.zeros((1,1024,15,20),ctx=mx.gpu())]))
feature_executor = pool5.bind(ctx=mx.gpu(),args=rois_dict)
feature_extractor = namedtuple("feature_extracter", ["executor","rois","fm","features"])
fexecutor = feature_extractor(executor = feature_executor,
                            rois = rois_dict["rois"],
                             fm = rois_dict["fm"],
                             features = feature_executor.outputs[0])

['fm', 'rois']


In [14]:
def prepocessd_image(raw_img, data_mean):
    img_processed = image_to_h5_mx(raw_img, data_mean, image_scaling=1.0)
    return {"raw": raw_img, "image": img_processed}

def load_video_file(video_file, data_mean):
    cap = cv2.VideoCapture(video_file)
    while not cap.isOpened():
        cap = cv2.VideoCapture(video_file)
        cv2.waitKey(1000)
        print "Wait for the header"

    pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
    print "frame count", cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
    while True:
        flag, frame = cap.read()
        if flag:
            pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
            raw_img = frame[-480:, :640, :]
            
            res = prepocessd_image(raw_img, data_mean)
            res['frame_no'] = pos_frame
            yield res

        else:
            # The next frame is not ready, so we try to read it again
            cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES, pos_frame-1)
            print "frame is not ready"
            # It is better to wait for a while for the next frame to be ready
            break

        if cv2.waitKey(10) == 27:
            break
        if cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES) == cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames,
            # we stop
            break
    cap.release()

# det_file = open("./tianmulu/det.txt")

# def image_generator(img_file, data_mean):
#     img_list = open(img_file).readlines()
    
#     frame = 1
#     for i in img_list:
#         img_pair = i.strip().split(" ")
#         raw_img = imread(img_pair[0])
#         im = image_to_h5_mx(raw_img, data_mean, image_scaling=1.0)
#         num = int(img_pair[1])
#         p = []
#         for j in range(num):
#             det = det_file.readline().strip()
#             p.append([int(x) for x in det.split(" ")])
#         yield {"image":im,"raw":raw_img, "proposal":p, "frame_no":frame}
#         frame = frame + 1

# def generate_rois2(batch_idx, rect, proposals):
# #     proposals =  []
# #     for rect in acc_rects:
#     left_topx = rect[0]
#     left_topy = rect[1]
#     proposals.append([batch_idx,left_topx,left_topy,left_topx+rect[2],left_topy+rect[3]])
#     return proposals


In [15]:
video_file = "./Video5.avi"
# img_file = "./tianmulu/img_list.txt"
input_gen = load_video_file(video_file, image_mean)

In [16]:
grid_width = 20
grid_height = 15
img_width = 640
img_height = 480
import numpy as np
num = 0

# img = input_gen.next()
# for img_set in input_gen:
#     img_set = input_gen.next()
#     if num%100 == 0:
# img = img_set["image"]
# anno = img_set["anno"]

#     count_anno += len(anno.rects)
#     print type(anno)
# writer = "ll.txt"
# f = open("features.txt","w")

# bbox_path = './tianmulu/bboxes/'
# feature_path = './tianmulu/features/'
log_file = "./tianmulu/bbox.txt"

def forward_test(im_dict):
#     im_input = image_to_h5_mx(img, image_mean, image_scaling=1.0)
    
    timeExcutor.data[:] = mx.nd.array(im_dict["image"],mx.gpu())
    timeExcutor.lstm_mem_seed[:] = mx.nd.zeros((300,250),mx.gpu())
    timeExcutor.lstm_hidden_seed[:] =  mx.nd.zeros((300,250),mx.gpu())

    timeExcutor.executor.forward()
    bbox_list = [timeExcutor.bbox[idx].asnumpy() for idx in range(max_len)]
    conf_list = [timeExcutor.conf[idx].asnumpy() for idx in range(max_len)]

    all_rects = [[[] for x in range(grid_width)] for y in range(grid_height)]
    pix_per_w = img_width/grid_width
    pix_per_h = img_height/grid_height
    for n in range(len(bbox_list)):
        for k in range(grid_height * grid_width):
            #print k,n,"k n"
            y = int(k / grid_width)
            x = int(k % grid_width)
            bbox = bbox_list[n][k]
            conf = conf_list[n][k,1].flatten()[0]
            abs_cx = pix_per_w/2 + pix_per_w*x + int(bbox[0,0,0])
            abs_cy = pix_per_h/2 + pix_per_h*y + int(bbox[1,0,0])
            w = bbox[2,0,0]
            h = bbox[3,0,0]
            #print x,y
            all_rects[y][x].append(Rect(abs_cx,abs_cy,w,h,conf))
    acc_rects = stitch_rects(all_rects)
#     print len(acc_rects)
    proposals = []
    img = im_dict["raw"]
    for rect in acc_rects:
        generate_rois(0, rect, proposals)
        cv2.rectangle(img, (proposals[-1][1],proposals[-1][2]),(proposals[-1][3],proposals[-1][4]),color=(0,0,255),thickness=2)
    imsave("./check2/%i.jpg"%int(im_dict["frame_no"]), img)
    fexecutor.fm[:] = timeExcutor.fm
    pos_frame = int(im_dict["frame_no"])
    l = open(log_file, "a")
    for p in proposals:
        fexecutor.rois[:] = mx.nd.array(p,mx.gpu()).reshape((1,5))
#         print p
        fexecutor.executor.forward()
        l.write(str(pos_frame)+' '+str(p[1])+' '+str(p[2])+' '+str(p[3])+' '+str(p[4])+' ')
        np.savetxt(l, fexecutor.features.asnumpy().reshape(1,-1),fmt = "%.8g")
    l.close()


In [17]:
def updatefig(*args):
    new_frame = forward_test(input_gen.next())
    im.set_array(new_frame)
    output_video.write(new_frame)
    return im

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import colormaps as cmaps
# fourcc = cv2.cv.CV_FOURCC(*'XVID')
# output_video = cv2.VideoWriter(video_file+'out.avi',fourcc, 20.0, (640,480))

# # output_label_path = './tianmulu/labels/'
# # output_image_path = './tianmulu/images/'
# # fig = 
# fig = plt.figure()
# im = plt.imshow(forward_test(input_gen.next()), animated=True)
# ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
# plt.show()
for i in range(1000):
    forward_test(input_gen.next())
    print i

frame count 52399.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [ ]:
# grid_width = 20
# grid_height = 15
# img_width = 640
# img_height = 480

# start = time.clock()
# num = 0

# img = input_gen.next()
# # for img_set in input_gen:
# #     img_set = input_gen.next()
# #     if num%100 == 0:
# # img = img_set["image"]
# # anno = img_set["anno"]

# #     count_anno += len(anno.rects)
# #     print type(anno)
# im_input = image_to_h5_mx(img, image_mean, image_scaling=1.0)

# timeExcutor.data[:] = mx.nd.array(im_input,mx.gpu())
# timeExcutor.lstm_mem_seed[:] = mx.nd.zeros((300,250),mx.gpu())
# timeExcutor.lstm_hidden_seed[:] =  mx.nd.zeros((300,250),mx.gpu())

# timeExcutor.executor.forward()

# bbox_list = [timeExcutor.bbox[idx].asnumpy() for idx in range(max_len)]
# conf_list = [timeExcutor.conf[idx].asnumpy() for idx in range(max_len)]

# all_rects = [[[] for x in range(grid_width)] for y in range(grid_height)]
# pix_per_w = img_width/grid_width
# pix_per_h = img_height/grid_height
# for n in range(len(bbox_list)):
#     for k in range(grid_height * grid_width):
#         #print k,n,"k n"
#         y = int(k / grid_width)
#         x = int(k % grid_width)
#         bbox = bbox_list[n][k]
#         conf = conf_list[n][k,1].flatten()[0]
#         abs_cx = pix_per_w/2 + pix_per_w*x + int(bbox[0,0,0])
#         abs_cy = pix_per_h/2 + pix_per_h*y + int(bbox[1,0,0])
#         w = bbox[2,0,0]
#         h = bbox[3,0,0]
#         #print x,y
#         all_rects[y][x].append(Rect(abs_cx,abs_cy,w,h,conf))

# # for rect in acc_rects:

# #     if rect.true_confidence < 0.6:
# #         continue
# acc_rects = stitch_rects(all_rects)
# proposals = []
# print len(acc_rects)

# for rect in acc_rects:
#     if rect.true_confidence < 0.6:
#         continue
#     generate_rois(0, rect, proposals)
#     cv2.rectangle(img, (rect.cx-int(rect.width/2), rect.cy-int(rect.height/2)),\
#      (rect.cx+int(rect.width/2), rect.cy+int(rect.height/2)),color=(0,0,255),thickness=2)
# print len(proposals)
# # print len(acc_rects)
# # fexecutor.rois[:] = mx.nd.array(generate_rois(0,acc_rects),mx.gpu())
# # print rois_dict["rois"].shape
# # # fexecutor.fm[:] = generate_rois(0,acc_rects)
# # rois_dict["fm"] = timeExcutor.fm

# # # for rect in acc_rects:

# #     if rect.true_confidence < 0.6:fm[:]
# #         continue
# #     cv2.rectangle(img, (rect.cx-int(rect.width/2), rect.cy-int(rect.height/2)),\
# #      (rect.cx+int(rect.width/2), rect.cy+int(rect.height/2)),color=(0,0,255),thickness=2)
# # imsave("./mxnet/%i.jpg"% i, img)
# # print time.clock() - start

In [24]:
# fexecutor.executor.forward()
# print fexecutor.features.shape
import numpy as np
fexecutor.fm[:] = timeExcutor.fm
writer = "ll.txt"
f = open(writer,"w")
for p in proposals:
    fexecutor.rois[:] = mx.nd.array(p,mx.gpu()).reshape((1,5))
    fexecutor.executor.forward()
    print np.max(fexecutor.features.asnumpy())
#     for a in 
    np.savetxt(f, fexecutor.features.asnumpy().reshape(1,-1),fmt = "%.8g")
    f.write("\n\n\n\n")
    print "haha"

NameError: name 'proposals' is not defined

In [25]:
b = mx.nd.array([1,2,3]).reshape((1,3))
# help (mx.)


In [26]:
b.shape

(1L, 3L)